In [9]:
import pickle

import pandas as pd
import numpy as np
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.palettes
import bokeh.plotting
import bokeh.transform
import bokeh.embed
import json

import sp_project.data_collection.openweather_prediction as weatherprediction
import sp_project.app_state
from sp_project.data_preparation.db_client import get_global_db_client
from sp_project.notebook_tools import default_clients
from sp_project.data_modelling.create_model import *
from sp_project.data_preparation.prediction_preparation import *

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
lon = 7.70
lat = 46.42

In [3]:
with open("../../src/sp_project/backend_server/assets/prediction-model.pickle", "rb") as fh:
    model = pickle.load(fh)

In [4]:
async with default_clients() as (OWclient, db):
    app_state = sp_project.app_state.AppState(ow_client=OWclient, db_client=db, model=model)
    result = await extract_predictions_daily(app_state, lon=lon, lat=lat)
    
    features = prepare_prediction_features(result, lat)
    prediction = energy_prediction(app_state.model, features)

In [5]:
prediction.head()

,wind,solar,nuclear,water_reservoir,water_river,water_pump,total
dt,,,,,,,
2023-05-12 11:00:00+00:00,28.105782,80.312142,2082.664081,623.648032,245.581582,514.133777,3569.112363
2023-05-13 11:00:00+00:00,31.231668,77.250621,2122.723859,550.789164,244.790689,475.217454,3497.357892
2023-05-14 11:00:00+00:00,34.236490,53.936871,2161.411440,476.836266,242.306921,443.096554,3407.496480
2023-05-15 11:00:00+00:00,40.122482,76.171577,2236.596924,345.026430,243.186627,362.549129,3301.091525
2023-05-16 11:00:00+00:00,41.861932,-0.268615,2259.546626,290.116589,236.423310,360.294970,3184.536057


In [11]:
prediction_source = bokeh.models.ColumnDataSource(prediction)

column_names = "nuclear solar wind water_reservoir water_pump water_river".split()
colors = "#D55E00 #F0E442 #BBBBBB #009E73 #0072B2 #56B4E9".split()

middle = len(prediction) // 2
selection_range = 2

# Selected-Part-Plot
p = bokeh.plotting.figure(
    height=300,
    width=800,
    x_axis_type="datetime",
    x_axis_location="above",
    x_range=(prediction.index[middle - selection_range], prediction.index[middle + selection_range]),
)

p.yaxis.axis_label = 'Energy Production [MW]'

p.varea_stack(
    x='dt',
    stackers=column_names,
    source=prediction_source,
    color=colors,
    legend_label=column_names,
)

# Overview-Plot
overview = bokeh.plotting.figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=130,
    width=800,
    x_axis_type="datetime",
    y_range=p.y_range,
    y_axis_type=None,
    tools="hover",
    toolbar_location=None,
)

overview.varea_stack(
    x='dt',
    stackers=column_names,
    source=prediction_source,
    color=colors,
)

p.legend.location = "top_left"

range_tool = bokeh.models.RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

overview.ygrid.grid_line_color = None
overview.add_tools(range_tool)

overview_fig = bokeh.layouts.column(p, overview)

bokeh.plotting.show(overview_fig)

In [7]:
x = round(prediction.iloc[:, :-1].sum(), 2)

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'country'})
data['angle'] = data['value'] / data['value'].sum() * 2 * np.pi
data['color'] = "#BBBBBB #F0E442 #D55E00 #009E73 #0072B2 #56B4E9".split()

p = bokeh.plotting.figure(
    height=350,
    title="Average Production for the next 8 Days",
    toolbar_location=None,
    tools="hover",
    tooltips="@country: @value",
    x_range=(-0.5, 1.0)
)

p.wedge(x=0, y=1, radius=0.4,
        start_angle=bokeh.transform.cumsum('angle', include_zero=True), end_angle=bokeh.transform.cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

pie_fig = bokeh.layouts.column(p)

bokeh.plotting.show(pie_fig)

### Save the Graphics as JSON-Files

In [12]:
with open("energy_prediction_overview.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(overview_fig), fh)

In [13]:
with open("energy_prediction_pieplot.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(pie_fig), fh)